### 3. polarity score를 구해 사전 구축한다.
* polarity score = P(Ngram|Hawkish) / P(Ngram|Dovish)

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
news_report_mpb = pd.read_csv('../datasets/2_nrm_callrate_merge.csv')
news_report_mpb = news_report_mpb.dropna()
news_report_mpb

,date,content,ngram,label
0,2005-05-01,"4월은 잔인했다. 어닝 시즌, 기업들이 내놓은 실적은 실망감만 불렀고 세자리수대를...","['잔인/NNG', '실적시즌/NNG', '기업/NNG', '내놓/VV', '실적/...",up
1,2005-05-01,뉴욕증시 투자자들이 시장의 방향을 정하지 못한채 하루하루 재료에 과민반응하고 있다...,"['시장/NNG', '방향/NNG', '정하/VV', '못하/VX', '하루하루/M...",up
2,2005-05-01,지난주(4월25~29일) 940선대에서 출발한 증시가 910선대에서 마감했다. `...,"['선대/NNG', '출발/NNG', '하/VV', '증시/NNG', '선대/NNG...",up
3,2005-05-01,은행들이 수수료 수입에 급급한 나머지 `일단 팔고 보자`는 식의 상품판매 경쟁에 ...,"['은행/NNG', '수수료/NNG', '수입/NNG', '급급/NNG', '일단/...",up
4,2005-05-01,박성제= 3월 중소기업 대출 연체율이 작년 동기에 비해서는 낮아졌지만 작년 말에...,"['중소기업/NNG', '대출/NNG', '연체율/NNG', '비해서/VV', '낮...",up
...,...,...,...,...
226781,2017-09-30,우성문 통신원 = 유럽중앙은행(ECB)이 점진적으로 부양책을 없애게 된다면 은행들...,"['ecb/NNG', 'ecb/NNG', '점진적/VAX', '부양책/NNG', '...",up
226782,2017-09-30,우성문 통신원 = 패트릭 하커 필라델피아 연방준비은행 총재가 12월 금리 인상 가...,"['금리/NNG', '인상/NNG', '가능성/NNG', '시사/NNG', '하/V...",up
226783,2017-09-30,이종혁 = 지난 8월 미국의 소비지출이 전월보다 둔화한 데다 물가는 계속 부진한...,"['소비/NNG', '지출/NNG', '둔화/NNG', '하/VV', '물가/NNG...",up
226784,2017-09-30,"신은실 = 뉴욕 금가격은 달러화 강세 영향 등으로 소폭 하락했다. , 29일 뉴...","['은실/NNG', '뉴욕/NNG', '금/NNG', '가격/NNG', '달러/NN...",up


In [3]:
# 데이터프레임에 있는 단어 빈도수 계산
def cnt_value(df, col_name):

    tot_cnt = df['ngram'].str.split(expand=True).stack().value_counts()
    tot_cnt = tot_cnt.to_frame(name=col_name)
    tot_cnt = tot_cnt.reset_index().rename(columns={"index": "ngram"})

    return tot_cnt


# 데이터프레임에 있는 단어 빈도수 계산
def cnt_value2(df, col_name):
    word_ls = []
    for content in df['ngram']:
        content = content.split(',')
        for con in content:
            con = con.replace("[","").replace("]","").replace("'","").replace(" ","")
            word_ls.append(con)

    word_freq = Counter(word_ls)
    freq_df = pd.DataFrame.from_records(list(dict(word_freq).items()), columns=['ngram', col_name])
    
    return freq_df

u_DF = news_report_mpb.loc[news_report_mpb['label']== 'up']
d_DF = news_report_mpb.loc[news_report_mpb['label']== 'down']

# up_count 컬럼 생성
up_df = cnt_value2(u_DF, 'up_count')
# down_count 컬럼 생성
down_df = cnt_value2(d_DF, 'down_count')

# up_count과 down_count을 병합한 후 더해서 total_count 컬럼 생성
up_down_df = pd.merge(up_df,down_df, on='ngram', how='outer')
up_down_df = up_down_df.fillna(0)
up_down_df['total_count'] = up_down_df['up_count']+up_down_df['down_count']

is_above_15 = up_down_df['total_count'] >= 15
up_down_total = up_down_df[is_above_15]
up_down_total

,ngram,up_count,down_count,total_count
0,잔인/NNG,38.0,39.0,77.0
1,실적시즌/NNG,1204.0,1168.0,2372.0
2,기업/NNG,43336.0,49020.0,92356.0
3,내놓/VV,11614.0,13944.0,25558.0
4,실적/NNG,25299.0,26869.0,52168.0
...,...,...,...,...
134897,適應/NNG,0.0,25.0,25.0
135179,금융시장/NNG;부정적/VAX;영향/NNG;우려/NNG,0.0,18.0,18.0
135929,탈퇴/NNG;우려/NNG;완화/NNG,0.0,18.0,18.0
137279,물가/NNG;상승/NNG;상향/NNG,0.0,15.0,15.0


In [4]:
# polar_score 식
up_count_sum = up_down_total['up_count'].sum()
down_count_sum = up_down_total['down_count'].sum()

p_ngram_dovish = up_down_total['down_count'] / down_count_sum
p_ngram_hawkish = up_down_total['up_count'] / up_count_sum

up_down_total['polar_score'] = p_ngram_hawkish / p_ngram_dovish

# polar_score > 1.3 : Hawkish / polar_score < 0.76 : Dovish / (1/1.3) <= polar_score <= 1.3 : nothing
up_down_total['polarity'] = np.where(up_down_total['polar_score'] > 1.3, 'Hawkish', np.where(up_down_total['polar_score'] < (1/1.3), 'Dovish', 'nothing'))
up_down_total

,ngram,up_count,down_count,total_count,polar_score,polarity
0,잔인/NNG,38.0,39.0,77.0,1.107190,nothing
1,실적시즌/NNG,1204.0,1168.0,2372.0,1.171351,nothing
2,기업/NNG,43336.0,49020.0,92356.0,1.004567,nothing
3,내놓/VV,11614.0,13944.0,25558.0,0.946450,nothing
4,실적/NNG,25299.0,26869.0,52168.0,1.069930,nothing
...,...,...,...,...,...,...
134897,適應/NNG,0.0,25.0,25.0,0.000000,Dovish
135179,금융시장/NNG;부정적/VAX;영향/NNG;우려/NNG,0.0,18.0,18.0,0.000000,Dovish
135929,탈퇴/NNG;우려/NNG;완화/NNG,0.0,18.0,18.0,0.000000,Dovish
137279,물가/NNG;상승/NNG;상향/NNG,0.0,15.0,15.0,0.000000,Dovish


In [5]:
# 긍정(Hawkish)/부정(Dovish)로 나눠서 사전 저장하기
pos_dict = up_down_total.query("polarity == 'Hawkish'")
neg_dict = up_down_total.query("polarity == 'Dovish'")

pos_dict.to_csv('../datasets/3_pos_dict.csv',index=False) 
pos_dict.to_csv('../datasets/3_neg_dict.csv',index=False) 